In [0]:
# 종목코드와 기업의 이름 두가지의 경우가 입력값으로 들어올 수 있음.
# int로 형변환을 했을 때 기업의 이름을 형변환 할 경우 오류가 발생하는 것을 이용하여 분류
# 오류가 발생하였을 경우 기업의 이름이 들어온 것이므로 네이버에 기업의 이름을 검색 하였을 때 <증권정보>를 찾아서 종목코드를 알아냄.
# url 디코드 인코드 참고: https://dololak.tistory.com/255


import requests
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd
from pandas import DataFrame as df

def change_stock_to_code(stock): 
  try : #종목코드로 들어온 것
    int(stock)     

  except : # 종목코드가 아닌 어떤 문자가 들어오게되면 오류가 발생    
    incode_stock=parse.quote(stock)
    search_url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query="
    search_url= search_url+incode_stock  
    stocks_url = requests.get(search_url)
    html = stocks_url.text
    soup = BeautifulSoup(html, 'html.parser')
    search = soup.find_all("script") #script 구문을 모두 search에 저장
    for i in range(0,len(search)): # 모든 script 구문을 돌아보며 sItemCode를 가진 script를 찾으면 break
      if 'sItemCode' in search[i].text:                  
        break
    
    # search[i] << 여기서 종목코드를 뽑아내야 함
    # string의 특정 단어 위치 찾기 https://hashcode.co.kr/questions/286/%EC%8A%A4%ED%8A%B8%EB%A7%81%EC%97%90-%ED%8A%B9%EC%A0%95-%EB%8B%A8%EC%96%B4-%EC%9C%84%EC%B9%98-%EC%B0%BE%EA%B8%B0
    # sItemCode의 index 찾기
    st1 = search[i].text
    st2 = "sItemCode"
    idx = st1.find(st2)
    if idx==-1:
        print('올바르지 않은 종목명 입니다.')
        return 0
    else:        
        stock = search[i].text[idx+13:idx+19] #종목코드       
        try:
          int(stock)
          return stock
        except:
          print("해당 증권정보는 종목코드가 없습니다.")
          return 0
          
    

  
def crawling(stock_code):
  URL = "https://finance.naver.com/item/main.nhn?code={}".format(stock_code)
  stocks_url = requests.get(URL)
  html = stocks_url.text
  soup = BeautifulSoup(html, 'html.parser')
  
  # 재무제표가 없는 선물주식등은 분석이 불가능 ("기업실적분석" 이라는 string의 유무로 재무제표의 유무를 판단가능)
  # 예외처리

  #크롤링 부분
  #종목시세부분
  print("종목 시세 정보")
  market_info=soup.find("dl","blind") #종목시세정보
  market_info=market_info.find_all("dd")
  for i in range(0,len(market_info)):
    print(market_info[i].get_text())

  print() #공백

  # 기업개요부분
  summary_info = soup.find("div","summary_info") #기업개요 부분
  summary_info=summary_info.find_all("p")
  for i in range(0,len(summary_info)):
    print(summary_info[i].get_text())

  print() #공백

  #재무제표부분
  annual_date = []
  quarter_date = []
  attribute = []
  save_data = []

  try : 
    cas=soup.find("div","section cop_analysis") #재무제표 부분 cas = cop_analysis_section
    cas_rc=cas.find_all("th") #rc = row_columns
    cas_data=cas.find_all("td")
  except : #재무제표가 없을경우
    print("해당 증권정보는 재무제표가 없습니다.")
    return 0
  

  for i in range(3,7): #split() 함수를 사용하여 \n\t 등 공백제거
    annual_date.append(cas_rc[i].get_text().split())

  for i in range(7,13):
    quarter_date.append(cas_rc[i].get_text().split())

  for i in range(23,39):
    attribute.append(cas_rc[i].get_text().split())

  for i in range(0,len(cas_data)):
    save_data.append(cas_data[i].get_text().split())



  #데이터를 모두 모았음으로 표를 만들면 됨 DataFrame 부분의 data= 뒷부분에 dictionary 가 필요함
  # DataFrame data 파트 만들기
  dict_data = dict()
  dict_list = []
  x = 0
  for i in range(0,len(attribute)):  
      dict_list.append([save_data[x],save_data[x+1],save_data[x+2],save_data[x+3],save_data[x+4],save_data[x+5],save_data[x+6],save_data[x+7],save_data[x+8],save_data[x+9]])
      x=x+(len(annual_date)+len(quarter_date)) # 년도와 분기의 개수 만큼 속성별 데이터묶음을 가져오도록 

  #DataFrame index부분 만들기
  index_list = [annual_date[0][0],annual_date[1][0],annual_date[2][0],annual_date[3][0],quarter_date[0][0],quarter_date[1][0],quarter_date[2][0],quarter_date[3][0],quarter_date[4][0],quarter_date[5][0]]


  for i in range(0,len(attribute)):  #dictionary 만들기
    dict_data[str(attribute[i])] = dict_list[i]


  Financial_Statements = df(data=dict_data ,index = index_list)
  print(Financial_Statements)
  return dict_list,market_info


def Financial_analysis(dict_list,market_info):
  if dict_list == 0 : #재무제표가 없는경우 dict_list =0
    return 0;
  else:
  #dict_list의 요소들이 str이기 때문에 크기비교를 할 수 없음. 따라서 int로 변환해야하는데 -와 , 를 반영해야 할것
    for i in range(0,len(dict_list)):
      for j in range(0,len((dict_list[0]))):
        try:      
          dict_list[i][j][0]=dict_list[i][j][0].replace(",","") # 문자열의 ,를 ""아무것도 없는것으로 바꿔줌      
        except:  #빈 리스트들을 불러오거나 print하면 에러가 발생함 아래에서 비교할때 재무제표별로 무슨항목이 비어있는지 모르기때문에 각각 예외처리를 해줘야 함
          continue
        

    result = 0

    #매출액 분석
    try:
      if(float(dict_list[0][0][0]) < float(dict_list[0][1][0]) and float(dict_list[0][1][0]) < float(dict_list[0][2][0])):
        print("매출액이 최근3년간 증가하고 있는 기업입니다.")
        result = result + 10
      elif(float(dict_list[0][0][0]) > float(dict_list[0][1][0]) and float(dict_list[0][1][0]) > float(dict_list[0][2][0])):
        print("매출액이 최근3년간 감소하고 있는 기업입니다.")
        result = result - 10
      else:
        result = result - 5
    except:
      print("매출액에 공백이 있어 분석할 수 없습니다.")

    #당기손이익
    try:
      if(float(dict_list[2][2][0]) < 0):
        ("최근년도에 적자를 기록한 기업입니다.")
        result = result - 100
      elif(float(dict_list[2][0][0]) < float(dict_list[2][1][0]) and float(dict_list[2][1][0]) < float(dict_list[2][2][0])):
        print("당기손이익이 최근3년간 증가하고 있는 기업입니다.")
        result = result + 20
      elif(float(dict_list[2][0][0]) > float(dict_list[2][1][0]) and float(dict_list[2][1][0]) > float(dict_list[2][2][0])):
        print("당기손이익이 최근3년간 감소하고 있는 기업입니다.")
        result = result - 20
      else:
        result = result - 10
    except:
      print("당기손이익에 공백이 있어 분석할 수 없습니다.")

    #부채비율 : 기업의 자기자본에 대한 부채의 상대적 크기로 기업의 재무위험을 나타내는 지표입니다. 100%이하 안정적 400%이상 조심 600%이상 위험 **은행,금융관련주들은 부채비율무시
    try:
      if(float(dict_list[6][2][0])<100):
        print("부채비율이 안정적인 기업입니다.")
      elif(float(dict_list[6][2][0])>400):
        print("부채비율을 조심해야할 기업입니다.")
      elif(float(dict_list[6][2][0])>600):
        print("부채비율이 위험한 기업입니다. *금융,은행관련주들은 제외")
    except:
      print("부채비율에 공백이 있어 분석할 수 없습니다.")

    #당좌비율 : 유동자산 중 가장 짧은 기간에 현금화가 가능한 당좌자산과 유동부채의 규모를 비교함으로써 기업의 단기지급능력을 나타내는 지표입니다.
    try:
      if(float(dict_list[7][2][0]) > 100): 
        print("기업의 단기지급능력(당좌비율)이 안정적입니다.")
    except:
      print("당좌비율에 공백이 있어 분석할 수 없습니다.")

    #유보율 : 기업이 동원할 수 있는 자금량을 측정하는 지표로, 이것이 높을수록 불황에 대한 적응력과 기업의 안전성이 높음. (자본 잉여금+이익 잉여금)/(납입 자본금) * 100
    try:
      if(float(dict_list[8][2][0]) > 100): 
        print("불황에 대한 적응력과 안전성(유보율)이 높은 기업입니다.")
    except:
      print("유보율에 공백이 있어 분석할 수 없습니다.")

    #PER = 순이익대비 주가가 얼마나 고평가 되었는지 나타내는 지표  (시가총액 / 당기순이익)
    try:
      if(float(dict_list[10][2][0]) > 10): 
        print("순이익대비주가(PER)가 고평가 되었습니다.")
    except:
      print("PER에 공백이 있어 분석할 수 없습니다.")

    current_price=market_info[3].get_text().split()[1].replace(",","")
    #BPS : 기업이 지금당장 모든 활동을 중당하고 기업의 자산을 주주들에게 나눠줄 경우, 한 주당 얼마씩 돌아가는지를 나타내는 지표(기업 순자산/발행주식수)
    try:
      if(float(dict_list[11][2][0]) > float(current_price)): 
        print("현재 발행된 주식자산보다 기업순자산이 더 많습니다.")
      else:
        print("기업순자산보다 현재 발행된 주식자산 더 많습니다.")
    except:
      print("BPS에 공백이 있어 분석할 수 없습니다.")


    #PBR = 기업자본 대비 주가가 얼마나 고평가 되었는지 나타내는 지표 PBR이 1이하이면 자본대비 저평가, 1이상이면 자본대비 고평가 (시가총액/ 자본총액)
    try:
      if(float(dict_list[12][2][0]) <= 1): 
        print("자본대비 저평가된 기업입니다.")
      else:
        print("자본대비 고평가된 기업입니다.")
    except:
      print("PBR에 공백이 있어 분석할 수 없습니다.")

    if result > 0:
      print("매수 추천")
      print("투자에 관한 책임은 투자자에게 있으며 단순한 추천으로 인한 매수는 손실을 가져올 수 있습니다.\n현금흐름표 등 추가적인 재무제표분석과 정밀분석을 통해 매수를 결정하시길 바랍니다.")
    else:
      print("매수 비추천")

    
    
def explain():
  print("위 매수 알고리즘은 당기 순이익의 연속적인 성장에 가장 큰 영향을 받습니다\n")
  print("용어 설명")
  print("영업이익 : 기업이 한 해 동안 영업을 통하여 벌어들인 수익")
  print("당기순이익 : 기업이 한 해 동안 벌어들인 총수익(매출액, 영업외수익)에서 총비용(매출원가, 판매비와관리비, 영업외비용, 법인세비용)을 차감한 최종 이익을 당기순이익이라고 한다.")
  print("ROE(자기자본이익률): 기업이 자본을 이용하여 얼마만큼의 이익을 냈는지를 나타내는 지표입니다. (당기순이익/자본총액)")
  print("부채비율 : 기업의 자기자본에 대한 부채의 상대적 크기로 기업의 재무위험을 나타내는 지표입니다.")
  print("당좌비율 : 유동자산 중 가장 짧은 기간에 현금화가 가능한 당좌자산과 유동부채의 규모를 비교함으로써 기업의 단기지급능력을 나타내는 지표입니다( 당좌자산/유동부채 ) × 100")
  print("유보율 : 기업이 동원할 수 있는 자금량을 측정하는 지표로, 이것이 높을수록 불황에 대한 적응력과 기업의 안전성이 높음.(자본 잉여금+이익 잉여금)/(납입 자본금) * 100")
  print("EPS : 주당순이익(당기순이익/발행주식수)")
  print("PER = 순이익대비 주가가 얼마나 고평가 되었는지 나타내는 지표(시가총액 / 당기순이익)")
  print("BPS : 기업이 지금당장 모든 활동을 중당하고 기업의 자산을 주주들에게 나눠줄 경우, 한 주당 얼마씩 돌아가는지를 나타내는 지표(기업 순자산/발행주식수)")
  print("PBR = 기업자본 대비 주가가 얼마나 고평가 되었는지 나타내는 지표 PBR이 1이하이면 자본대비 저평가, 1이상이면 자본대비 고평가(시가총액/ 자본총액)")





In [57]:
#설계
# https://github.com/Junuu/Financial-Statement-Analysis/issues/1
# 코드 참고 https://engkimbs.tistory.com/625

stock = input('종목코드 또는 종목명을 입력해주세요:')
stock = change_stock_to_code(stock) 
if stock == 0 : #올바르지않은 종목명을 검색하였거나 ,종목코드가없는 코스피200, wti 등을 검색한 경우 
  print(stock)  

else: # 재무제표가 없는 선물주식등은 분석이 불가능 ("기업실적분석" 이라는 string의 유무로 재무제표의 유무를 판단가능)
  dict_list,market_info=crawling(stock)
  print()
  Financial_analysis(dict_list,market_info)
  print()
  explain()
  
  


#골든크로스 종목도 크롤링해서 분석하구.. 

종목코드 또는 종목명을 입력해주세요:한국금융지주
종목 시세 정보
2020년 06월 01일 15시 01분 기준 장중
종목명 한국금융지주
종목코드 071050 코스피
현재가 54,900 전일대비 상승 2,200 플러스 4.17 퍼센트
전일가 52,700
시가 52,900
고가 55,100
상한가 68,500
저가 52,900
하한가 36,900
거래량 367,081
거래대금 19,966백만

2003년 1월 설립되었으며, 금융투자, 자산운용, 인터넷전문은행, 저축은행, 벤처/PEF투자, 여신전문업, 헤지펀드 운용 등 금융업 전반의 영역에서 상품과 서비스를 제공하고 있음.
동사는 2017년 4월 자회사 한국카카오은행의 은행업 인가에 따라 은행지주로 전환됨.
동사는 한국투자증권, 한국투자저축은행, 한국투자캐피탈, 한국투자부동산 등 투자금융계열사들을 자회사로 두고 있음. 2020년 1월 카카오은행은 계열회사 제외되었음.

              ['매출액'] ['영업이익'] ['당기순이익']  ... ['주당배당금(원)'] ['시가배당률(%)'] ['배당성향(%)']
2017.12      [66,220]  [6,543]   [4,631]  ...      [1,600]       [2.32]     [18.38]
2018.12      [84,521]  [6,427]   [5,159]  ...      [1,800]       [3.03]     [19.92]
2019.12     [107,713]  [9,418]   [8,472]  ...      [2,900]       [4.01]     [20.08]
2020.12(E)         []  [5,536]   [4,585]  ...      [1,785]           []          []
2018.12      [23,317]    [315]      [-8]  ...           []           []          []
2019.03      [34,257]  [3,086]   [2

매출액이 최근3년간 증가하고 있는 기업입니다.
당기손이익이 최근3년간 증가하고 있는 기업입니다.
부채비율을 조심해야할 기업입니다.
당좌비율에 공백이 있어 분석할 수 없습니다.
불황에 대한 적응력과 안전성(유보율)이 높은 기업입니다.
현재 발행된 주식자산보다 기업순자산이 더 많습니다.
자본대비 저평가된 기업입니다.
매수 추천


30